In [ ]:
import pickle
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import uproot_methods
import networkx as nx

In [ ]:
g = pickle.load(open("graph_1.pkl", "rb"))

In [ ]:
def has_reco(g, particle):
    for edge in g.edges(particle):
        if edge[1][0] in ["tower", "track"]:
            return True
    return False

In [ ]:
particles = [n for n in g.nodes if n[0] == "particle"]
particles_with_reco = [n for n in particles if has_reco(g, n)]

In [ ]:
etas = np.array([g.nodes[n]["eta"] for n in particles_with_reco])
phis = np.array([g.nodes[n]["phi"] for n in particles_with_reco])
energys = np.array([g.nodes[n]["energy"] for n in particles_with_reco])

etas_tower = np.array([g.nodes[n]["eta"] for n in g.nodes if n[0] == "tower"])
phis_tower = np.array([g.nodes[n]["phi"] for n in g.nodes if n[0] == "tower"])
energys_tower = np.array([g.nodes[n]["energy"] for n in g.nodes if n[0] == "tower"])

etas_track = np.array([g.nodes[n]["eta"] for n in g.nodes if n[0] == "track"])
phis_track = np.array([g.nodes[n]["phi"] for n in g.nodes if n[0] == "track"])
energys_track = np.array([g.nodes[n]["pt"] for n in g.nodes if n[0] == "track"])

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(etas, phis, alpha=0.5, marker=".", label="GenParticle")
plt.scatter(etas_tower, phis_tower, alpha=0.5, marker="s", label="Tower")
plt.scatter(etas_track, phis_track, marker="x", label="Track")

plt.ylim(-4,4)
plt.legend(frameon=False)
plt.xlabel("eta")
plt.ylabel("phi")

In [ ]:
matrices = pickle.load(open("out.pkl", "rb"))

#this pred file is created by the training script
pred = np.load("experiments/run_01/pred.npz")
yp = pred["y_pred"]

In [ ]:
ypreds = []
ycands = []
ygens = []
for i in range(len(matrices["X"])):
    padval = 5000
    ycand = matrices["ycand"][i][:padval]
    ygen = matrices["ygen"][i][:padval]
    ycands.append(ycand)
    ygens.append(ygen)
    ypreds.append(yp[i, :ycand.shape[0]])
    
ygen = np.concatenate(ygens, axis=0)
ycand = np.concatenate(ycands, axis=0)
ypred = np.concatenate(ypreds, axis=0)
ypred_id = np.expand_dims(np.argmax(ypred[:, :6], axis=-1), -1)
ypred_charge = ypred[:, 6:7]
ypred_p4 = ypred[:, 7:]
print(ypred_id.shape, ypred_charge.shape, ypred_p4.shape)

ypred = np.concatenate([ypred_id, ypred_charge, ypred_p4], axis=-1)

In [ ]:
rng = np.linspace(0, 20, 100)
pid = 1
var_idx = 2

pid_names = {
    1: "charged hadrons",
    2: "neutral hadrons",
    3: "photons",
    4: "electrons",
    5: "muons"
}
var_names = {
    2: "pT",
    3: "eta",
    4: "phi",
    5: "E"
}

msk = (ygen[:, 0]==pid) & (ypred[:, 0]==pid)

plt.figure(figsize=(5,5))
plt.title("{} regression, {}".format(var_names[var_idx], pid_names[pid]))
plt.hist2d(
    ygen[msk, var_idx],
    ypred[msk, var_idx],
    bins=(rng, rng),
    cmap="Blues",
    norm=matplotlib.colors.LogNorm()
);
plt.xlabel("Gen {}".format(var_names[var_idx]))
plt.ylabel("MLPF {}".format(var_names[var_idx]))

msk = (ygen[:, 0]==pid) & (ycand[:, 0]==pid)
plt.figure(figsize=(5,5))
plt.title("{} regression, {}".format(var_names[var_idx], pid_names[pid]))
plt.hist2d(
    ygen[msk, var_idx],
    ycand[msk, var_idx],
    bins=(rng, rng),
    cmap="Blues",
    norm=matplotlib.colors.LogNorm()
);
plt.xlabel("Gen {}".format(var_names[var_idx]))
plt.ylabel("DelphesPF {}".format(var_names[var_idx]))

In [ ]:
pid = 1
var_idx = 2

def plot_reso(pid, var_idx, rng):
    msk = (ygen[:, 0]==pid) & (ypred[:, 0]==pid) & (ycand[:, 0]==pid)
    bins = np.linspace(-rng, rng, 100)
    yg = ygen[msk, var_idx]
    yp = ypred[msk, var_idx]
    yc = ycand[msk, var_idx]
    ratio_mlpf = (yp - yg) / yg
    ratio_dpf = (yc - yg) / yg
    ratio_mlpf[yg==0] = -1
    ratio_dpf[yg==0] = -1

    plt.figure(figsize=(4,4))
    ax = plt.axes()
    plt.title("{} resolution for {}".format(var_names[var_idx], pid_names[pid]))
    plt.hist(ratio_mlpf, bins=bins, histtype="step", lw=2, density=1.0, label="MLPF\n$\mu={:.1f},\\ \sigma={:.1f}$".format(np.mean(ratio_mlpf), np.std(ratio_mlpf)));
    plt.hist(ratio_dpf, bins=bins, histtype="step", lw=2, density=1.0, label="DelphesPF\n$\mu={:.1f},\\ \sigma={:.1f}$".format(np.mean(ratio_dpf), np.std(ratio_dpf)));
    plt.legend(frameon=False)
    plt.ylim(0, ax.get_ylim()[1]*2)
    
plot_reso(2, 5, 2)
plot_reso(2, 3, 0.5)
plot_reso(2, 4, 0.5)

In [ ]:
np.std(ratio_mlpf), np.std(ratio_dpf)

In [ ]:
plt.figure(figsize=(10,10))

X = matrices["X"][0]
ygen = matrices["ygen"][0]
ycand = matrices["ycand"][0]
plt.scatter(ygen[:, 3], ygen[:, 4], marker=".")
plt.scatter(ycand[:, 3], ycand[:, 4], marker="x")
plt.scatter(X[:, 2], X[:, 3], marker="s", alpha=0.5)


In [ ]:
msk = (ygen[:, 0] == 1) & (ycand[:, 0] == 1)
plt.figure(figsize=(5,5))
plt.hist((ygen[msk, 2]-ycand[msk, 2])/ygen[msk, 2], bins=np.linspace(-5,5,100));
plt.xlabel("pT resolution (gen-pf)/gen")
plt.title("Charged hadrons")

In [ ]:
msk = (ygen[:, 0] == 2) & (ycand[:, 0] == 2)
plt.figure(figsize=(5,5))
plt.hist( (ygen[msk, 5]-ycand[msk, 5])/ygen[msk, 5], bins=np.linspace(-5,5,100));
plt.xlabel("E resolution (gen-pf)/gen")
plt.title("neutral hadrons")

plt.figure(figsize=(5,5))
plt.hist((ygen[msk, 3]-ycand[msk, 3])/ygen[msk, 3], bins=np.linspace(-1,1,100));
plt.xlabel("eta resolution (gen-pf)/gen")
plt.title("Charged hadrons")

plt.figure(figsize=(5,5))
plt.hist((ygen[msk, 4]-ycand[msk, 4])/ygen[msk, 4], bins=np.linspace(-1,1,100));
plt.xlabel("phi resolution (gen-pf)/gen")
plt.title("Charged hadrons")

In [ ]:
msk = (ygen[:, 0] == 3) & (ycand[:, 0] == 3)
plt.figure(figsize=(5,5))
plt.hist( (ygen[msk, 5]-ycand[msk, 5])/ygen[msk, 5], bins=np.linspace(-5,5,100));
plt.xlabel("E resolution (gen-pf)/gen")
plt.title("photons")

In [ ]:
msk = (ygen[:, 0] == 4) & (ycand[:, 0] == 4)
plt.figure(figsize=(5,5))
plt.hist( (ygen[msk, 2]-ycand[msk, 2])/ygen[msk, 2], bins=np.linspace(-0.5, 0.5, 100))
plt.xlabel("pT resolution (gen-pf)/gen")
plt.title("muons")

In [ ]:
msk = (ygen[:, 0] == 5) & (ycand[:, 0] == 5)
plt.figure(figsize=(5,5))
plt.hist( (ygen[msk, 2]-ycand[msk, 2])/ygen[msk, 2], bins=np.linspace(-0.5, 0.5, 100))
plt.xlabel("pT resolution (gen-pf)/gen")
plt.title("electrons")

In [ ]:
import sklearn
import sklearn.metrics

In [ ]:
confusion = sklearn.metrics.confusion_matrix(
    ygen[:, 0], ycand[:, 0], normalize="true"
)

confusion2 = sklearn.metrics.confusion_matrix(
    ygen[:, 0], ypred[:, 0], normalize="true"
)

In [ ]:
plt.imshow(confusion, cmap="Blues")
plt.title("Reconstructed PID (normed to gen)")
plt.xlabel("Delphes PF PID")
plt.ylabel("Gen PID")
plt.xticks(range(6), ["none", "ch.had", "n.had", "g", "mu", "el"]);
plt.yticks(range(6), ["none", "ch.had", "n.had", "g", "mu", "el"]);
plt.colorbar()

In [ ]:
plt.imshow(confusion2, cmap="Blues")
plt.title("Reconstructed PID (normed to gen)")
plt.xlabel("MLPF PID")
plt.ylabel("Gen PID")
plt.xticks(range(6), ["none", "ch.had", "n.had", "g", "mu", "el"]);
plt.yticks(range(6), ["none", "ch.had", "n.had", "g", "mu", "el"]);
plt.colorbar()